In [ ]:
from pathlib import Path
import xarray as xr
from eopf import EOProduct
from eopf.common.constants import OpeningMode


In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
product = SAMPLE_PATH / "S3SLSRBT_20230315T095847_0179_B150_S819.zarr/"

In [ ]:
prod=EOProduct("prod",url=str(product)).open(mode=OpeningMode.OPEN)
prod.measurements.inadir.s7_bt_in


In [ ]:
group = "measurements/inadir"
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr",chunks={})
ds


In [ ]:
import matplotlib.pyplot as plt
bt=ds.s7_bt_in
plt.figure(figsize=(8, 8))
ax = plt.axes()
bt.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

In [ ]:
group = "conditions/meteo"
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr")
ds.temperature_profile_tp.interp(p_atmos=980).plot()